# Interpreter

> Converting expressions into OOP structures

We will build our own custom interpreter in this lesson. We will not be building something like a full Python interpreter, but rather something smaller that shows the basics.

Interpreters do their thing with 2 separate processes:
* **Lexing**: converting a string into separate lexical tokens
* **Parsing**: interpreting a sequence of tokens

## Lexing

We will begin with **lexing**: we want to convert the string `(13+4)-(12+1)` into its tokens:
1. We want a method (we will call it `calc`) that takes a string and converts it into an arithmetic operation represented by tokens.
2. Within this method, we will **lex** the input and convert it into **tokens**.
3. We will need to define what a **token** is as well as the `lex` method.

This translates to this skeleton code:

In [1]:
class Token:
    pass # we'll implement this later

def lex(input):
    pass # we'll implement this later

def calc(input):
    tokens = lex(input)
    
calc('(13+4)-(12+1)')

Let's implement our `Token` class. A token is any of the symbols that we can see in our arithmetic expression, so we will need to define every single one of those. Here's an implementation with enums:

In [2]:
from enum import Enum, auto

class Token:
    class Type(Enum): # the types of tokens that we can see in our expression
        INTEGER = auto()
        PLUS = auto()
        MINUS = auto()
        LPAREN = auto()
        RPAREN = auto()

    def __init__(self, type, text):
        """We keep both the type and the original string so that we can print the token"""
        self.type = type
        self.text = text

    def __str__(self):
        return f'`{self.text}`' # The backwards quotes `` are useful for delimiting characters such as white spaces

Now we will build our lexing process. We will create a `lex` function that will build up an array of tokens based on our string input, so we will have to loop through our string, convert each character into a token sequentially and append the tokens to the array: 

In [3]:
def lex(input):
    result = []

    i = 0
    while i < len(input):
        if input[i] == '+':
            result.append(Token(Token.Type.PLUS, '+'))
        elif input[i] == '-':
            result.append(Token(Token.Type.MINUS, '-'))
        elif input[i] == '(':
            result.append(Token(Token.Type.LPAREN, '('))
        elif input[i] == ')':
            result.append(Token(Token.Type.RPAREN, ')'))
        else:  # must be a number
            digits = [input[i]]
            for j in range(i + 1, len(input)):
                if input[j].isdigit():
                    digits.append(input[j])
                    i += 1
                else:
                    result.append(Token(Token.Type.INTEGER,
                                        ''.join(digits)))
                    break
        i += 1

    return result

The loop is imple enough for non-digits: we detect the token, append it to the array, increase the counter and go to the next character.

When we detect a digit, we can't follow the same strategy because the number twelve is a single token rather than a combination of two tokens (`12` is not the same as `1` and `2`), so what we do instead is create a secondary array where we append each digit that we detect until we detect a non-digit, then we create a single token for the detected number in the array.

Let's rerun our `calc` definition and test it:

In [4]:
def calc(input):
    tokens = lex(input)
    print(' '.join(map(str, tokens))) # we add spaces so that we can clearly tell the tokens apart
    
calc('(13+4)-(12+1)')

`(` `13` `+` `4` `)` `-` `(` `12` `+` `1` `)`


We can clearly see that each token has been properly detected, including multi-digit numbers.

Let's move on to **parsing**.

## Parsing

**Parsing** consists of turning a sequence of tokens into an **expression tree**, which we can subsequently traverse with the **visitor pattern** in order to either print or evaluate the expression.

We will create a `parse` function that will return this expression tree and we will add it to `calc`, but first, we need to define what the expression tree is made of. We will define 2 elements that can go in our tree: an `Integer` and a `BinaryExpression`, an expression with a left part and a right part; the expression can be either a subtraction or an addition:

In [5]:
class Integer:
    """We simply store the value of the integer"""
    def __init__(self, value):
        self.value = value
        
class BinaryExpression:
    class Type(Enum):
        ADDITION = auto()
        SUBTRACTION = auto()
    
    def __init__(self):
        self.type = None
        self.left = None
        self.right = None
        
    @property # getter
    def value(self):
        if self.type == self.Type.ADDITION:
            return self.left.value + self.right.value
        elif self.type == self.Type.SUBTRACTION:
            return self.left.value - self.right.value

We define a *property* in `BinaryExpression` that calculates the value of the expression based on its left and right sides. In other words, it **evaluates** the expression.

We will now define a `parse` function that will take a list of tokens as input. We will assume that the top level expression that we're going to build is always going to be a binary expression, for simplicity's sake. In other words: the return of this method will be of type `BinaryExpression`. The function will loop through the token list and decide what to do depending on the token type encountered:

In [6]:
def parse(tokens):
    result = BinaryExpression()
    have_lhs = False # lhs = left hand side
    i = 0
    while i < len(tokens):
        token = tokens[i]

        if token.type == Token.Type.INTEGER:
            integer = Integer(int(token.text))
            if not have_lhs:
                result.left = integer
                have_lhs = True
            else:
                result.right = integer
        elif token.type == Token.Type.PLUS:
            result.type = BinaryExpression.Type.ADDITION
        elif token.type == Token.Type.MINUS:
            result.type = BinaryExpression.Type.SUBTRACTION
        elif token.type == Token.Type.LPAREN:  # note: no if for RPAREN
            j = i
            while j < len(tokens):
                if tokens[j].type == Token.Type.RPAREN:
                    break
                j += 1
            # preprocess subexpression
            subexpression = tokens[i + 1:j]
            element = parse(subexpression)
            if not have_lhs:
                result.left = element
                have_lhs = True
            else:
                result.right = element
            i = j  # we skip the subexpression and proceed to the next token
        i += 1
    return result

Here's what's going on in the loop:

1. If we encounter an integer, we need to figure out whether it's the left or right side of the expression; we use a boolean flag for this.
2. If we encounter a plus or minus sign, we assign the expression type.
3. If we encounter an opening parenthesis, we need to detect the subexpression within, so we create another loop to detect all the tokens until we find the closing parenthesis and we recursively call `parse` on the subexpression; we then assign the subexpression to the left or right side according to the flag.

Let's expand `calc` with our parsing funtion and see how it works:

In [7]:
def calc(input):
    # lexing
    tokens = lex(input)
    print(' '.join(map(str, tokens)))
    
    # parsing
    parsed = parse(tokens)
    print(f'{input} = {parsed.value}')
    
calc('(13+4)-(12+1)')

`(` `13` `+` `4` `)` `-` `(` `12` `+` `1` `)`
(13+4)-(12+1) = 4


It works! We have successfully interpreted a string into an actual arithmetic expression.

> This implementation has many limitations: it does not support n-ary expressions such as `1+2+(3-4)` and it does not parse deep parenthesis nesting such as `(13+(2+2))-((6+6)+1)`. This is just a simple example for illustration purposes.